<a href="https://colab.research.google.com/github/tylerb55/COMP530/blob/main/ImageNet_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/tylerb55/COMP530.git

Cloning into 'COMP530'...
remote: Enumerating objects: 4216, done.
remote: Total 4216 (delta 0), reused 0 (delta 0), pack-reused 4216
Receiving objects: 100% (4216/4216), 914.09 MiB | 20.83 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
import numpy as np
import pandas as pd
import collections
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import PIL.Image as Image

# **Split the train test and validation set**

In [ ]:
def Train_Validation_Test_Set(directory_name):
  """a function to load the images in a large directory into a tensorflow dataset object
  the data is split 80:10:10 in training:validation:test. The dataset is shuffled before splitting
  and the images are formatted to 512x512 pixels and grayscale(one color channel and the values range from 0 to 255)"""
  dataset_train=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='training',
                                                                    batch_size=None
                                                                    )

  dataset_validation=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
  
  dataset_test=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.1,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
    
  dataset_validation=dataset_validation.take(dataset_test.__len__())

  return dataset_train,dataset_validation,dataset_test

In [ ]:
def ResNetPreprocess(image,label):
  image = image/255.0
  # Resize images from 512x512 to 224x224
  image = tf.image.resize(image, (224,224))
  return image, label

In [ ]:
BATCH_SIZE=32

dataset_train,dataset_validation, dataset_test=Train_Validation_Test_Set("Dataset1")
dataset_train=dataset_train.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_validation=dataset_validation.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_test=dataset_test.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)

AUTOTUNE = tf.data.AUTOTUNE
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_validation = dataset_validation.cache().prefetch(buffer_size=AUTOTUNE)

Found 1097 files belonging to 2 classes.
Using 878 files for training.
Found 1097 files belonging to 2 classes.
Using 219 files for validation.
Found 1097 files belonging to 2 classes.
Using 109 files for validation.


# **mobilenet v2**

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

num_classes = 1

mobilenet = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes,activation='sigmoid')
])

mobilenet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
mobilenet.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = mobilenet.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/30
27/27 [==============================] - 20s 108ms/step - loss: 0.5383 - binary_accuracy: 0.7373 - true_positives: 463.0000 - false_positives: 153.0000 - true_negatives: 174.0000 - false_negatives: 74.0000 - val_loss: 0.4171 - val_binary_accuracy: 0.7917 - val_true_positives: 50.0000 - val_false_positives: 12.0000 - val_true_negatives: 26.0000 - val_false_negatives: 8.0000
Epoch 2/30
27/27 [==============================] - 1s 40ms/step - loss: 0.3438 - binary_accuracy: 0.8692 - true_positives: 470.0000 - false_positives: 46.0000 - true_negatives: 281.0000 - false_negatives: 67.0000 - val_loss: 0.3717 - val_binary_accuracy: 0.8125 - val_true_positives: 49.0000 - val_false_positives: 9.0000 - val_true_negatives: 29.0000 - val_false_negatives: 9.0000
Epoch 3/30
27/27 [==============================] - 1s 40ms/step - loss: 0.3050 - binary_accuracy: 0.8808 - true_positives: 471.0000 - false_positives: 37.0000 - true_negatives: 290.0000 - false_negatives: 66.0000 - val_loss: 0.34

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

KeyError: ignored

# **Resnet 50**

In [ ]:
resnet = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
feature_extractor_model = resnet

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

num_classes = 1

resnet50 = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes,activation='sigmoid')
])

resnet50.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
resnet50.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 10

history = resnet50.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
resnet50.evaluate(dataset_test)

In [ ]:
mobilenet.evaluate(dataset_test)